# Introduction

**Name:** Meet Goel

**Student_ID:** GH1035975

**Module:** M516A Business Project in Big Data & AI

## Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import ast
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
import scipy.sparse
from scipy.sparse import load_npz
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/meetgoel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Reading the Dataset

In [2]:
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)


df = pd.read_csv('/Users/meetgoel/Downloads/MeetGG/Personal/Project/Assets/TMDB.csv')

df.shape

(10000, 23)

## Exploratory Data Analysis (EDA)

In [3]:
df.head(5)

,Unnamed: 0,movie_id,title,overview,keywords,genres,release_date,original_language,original_title,production_companies,revenue,status,tagline,spoken_languages,runtime,budget,vote_count,vote_average,popularity,adult_tf,poster_path,cast,crew
0,1,76600,Avatar: The Way of Water,Set more than a decade after the events of the...,"[{'id': 697, 'name': 'loss of loved one'}, {'i...","[{'id': 878, 'name': 'Science Fiction'}, {'id'...",2022-12-14,en,Avatar: The Way of Water,"[{'id': 574, 'logo_path': '/iB6GjNVHs5hOqcEYt2...",1443938691,Released,Return to Pandora.,"[{'english_name': 'English', 'iso_639_1': 'en'...",192,460000000,3655,7.711,4549.057,False,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,"[{'adult': False, 'gender': 2, 'id': 65731, 'k...","[{'adult': False, 'gender': 2, 'id': 1721, 'kn..."
1,2,661374,Glass Onion: A Knives Out Mystery,World-famous detective Benoit Blanc heads to G...,"[{'id': 703, 'name': 'detective'}, {'id': 1200...","[{'id': 9648, 'name': 'Mystery'}, {'id': 53, '...",2022-11-23,en,Glass Onion: A Knives Out Mystery,"[{'id': 37871, 'logo_path': '/lZW4kQSSLFldUpjj...",13280000,Released,NaN,"[{'english_name': 'English', 'iso_639_1': 'en'...",140,40000000,2204,7.083,3609.063,False,/vDGr1YdrlfbU9wxTOdpf3zChmv9.jpg,"[{'adult': False, 'gender': 2, 'id': 8784, 'kn...","[{'adult': False, 'gender': 2, 'id': 1226, 'kn..."
2,3,899112,Violent Night,When a team of mercenaries breaks into a wealt...,"[{'id': 65, 'name': 'holiday'}, {'id': 1991, '...","[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",2022-11-30,en,Violent Night,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...",73000000,Released,You better watch out.,"[{'english_name': 'English', 'iso_639_1': 'en'...",112,20000000,805,7.742,3283.632,False,/nhXjUvOvq7rJlvJFCrZMUUJ9Mn0.jpg,"[{'adult': False, 'gender': 2, 'id': 35029, 'k...","[{'adult': False, 'gender': 2, 'id': 6114, 'kn..."
3,4,411,"The Chronicles of Narnia: The Lion, the Witch ...","Siblings Lucy, Edmund, Susan and Peter step th...","[{'id': 83, 'name': 'saving the world'}, {'id'...","[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",2005-12-07,en,"The Chronicles of Narnia: The Lion, the Witch ...","[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",745013115,Released,Evil Has Reigned For 100 Years...,"[{'english_name': 'English', 'iso_639_1': 'en'...",143,180000000,9108,7.108,3057.713,False,/iREd0rNCjYdf5Ar0vfaW32yrkm.jpg,"[{'adult': False, 'gender': 2, 'id': 5527, 'kn...","[{'adult': False, 'gender': 2, 'id': 1095, 'kn..."
4,5,436270,Black Adam,"Nearly 5,000 years after he was bestowed with ...","[{'id': 952, 'name': 'lightning'}, {'id': 2095...","[{'id': 28, 'name': 'Action'}, {'id': 14, 'nam...",2022-10-19,en,Black Adam,"[{'id': 12, 'logo_path': '/iaYpEp3LQmb8AfAtmTv...",389000000,Released,The world needed a hero. It got Black Adam.,"[{'english_name': 'Arabic', 'iso_639_1': 'ar',...",125,200000000,3544,7.216,2590.202,False,/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg,"[{'adult': False, 'gender': 2, 'id': 18918, 'k...","[{'adult': False, 'gender': 2, 'id': 3687, 'kn..."


In [4]:
df = df[['overview', 'title', 'genres', 'keywords', 'cast', 'crew', 'movie_id']]

In [5]:
df.isnull().sum()

overview    85
title        0
genres       0
keywords     0
cast         0
crew         0
movie_id     0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   overview  9915 non-null   object
 1   title     10000 non-null  object
 2   genres    10000 non-null  object
 3   keywords  10000 non-null  object
 4   cast      10000 non-null  object
 5   crew      10000 non-null  object
 6   movie_id  10000 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


## Data Preprocessing

In [7]:
df.duplicated().sum()

1039

In [8]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.shape

(8887, 7)

In [9]:
df.iloc[0].genres

"[{'id': 878, 'name': 'Science Fiction'}, {'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}]"

In [10]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [11]:
df['genres'] = df['genres'].apply(convert)
df['keywords'] = df['keywords'].apply(convert)

In [12]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 5:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [13]:
df['cast'] = df['cast'].apply(convert3)

In [14]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [15]:
df['crew'] = df['crew'].apply(fetch_director)

In [16]:
df.head(5)

,overview,title,genres,keywords,cast,crew,movie_id
0,Set more than a decade after the events of the...,Avatar: The Way of Water,"[Science Fiction, Adventure, Action]","[loss of loved one, dying and death, alien lif...","[Sam Worthington, Zoe Saldaña, Sigourney Weave...",[James Cameron],76600
1,World-famous detective Benoit Blanc heads to G...,Glass Onion: A Knives Out Mystery,"[Mystery, Thriller, Comedy]","[detective, greece, investigation, satire, seq...","[Daniel Craig, Edward Norton, Janelle Monáe, K...",[Rian Johnson],661374
2,When a team of mercenaries breaks into a wealt...,Violent Night,"[Action, Comedy, Crime, Thriller]","[holiday, santa claus, mercenary, saving chris...","[David Harbour, John Leguizamo, Beverly D'Ange...",[Tommy Wirkola],899112
3,"Siblings Lucy, Edmund, Susan and Peter step th...","The Chronicles of Narnia: The Lion, the Witch ...","[Adventure, Family, Fantasy]","[saving the world, sibling relationship, witch...","[Skandar Keynes, Georgie Henley, William Mosel...",[Andrew Adamson],411
4,"Nearly 5,000 years after he was bestowed with ...",Black Adam,"[Action, Fantasy, Science Fiction]","[lightning, anti hero, superhero, based on com...","[Dwayne Johnson, Aldis Hodge, Noah Centineo, S...",[Jaume Collet-Serra],436270


In [17]:
df['overview'] = df['overview'].apply(lambda x:x.split())

In [18]:
df['genres'] = df['genres'].apply(lambda x:[i.replace(" ","") for i in x])
df['keywords'] = df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
df['cast'] = df['cast'].apply(lambda x:[i.replace(" ","") for i in x])
df['crew'] = df['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [19]:
df['tags'] = df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['crew']

In [20]:
df.head(1)

,overview,title,genres,keywords,cast,crew,movie_id,tags
0,"[Set, more, than, a, decade, after, the, event...",Avatar: The Way of Water,"[ScienceFiction, Adventure, Action]","[lossoflovedone, dyinganddeath, alienlife-form...","[SamWorthington, ZoeSaldaña, SigourneyWeaver, ...",[JamesCameron],76600,"[Set, more, than, a, decade, after, the, event..."


In [21]:
df2 = df[['movie_id', 'title', 'tags']]
df2.shape

(8887, 3)

In [22]:
df2['tags'] = df2['tags'].apply(lambda x: " ".join(x))

In [23]:
df2.head()

,movie_id,title,tags
0,76600,Avatar: The Way of Water,Set more than a decade after the events of the...
1,661374,Glass Onion: A Knives Out Mystery,World-famous detective Benoit Blanc heads to G...
2,899112,Violent Night,When a team of mercenaries breaks into a wealt...
3,411,"The Chronicles of Narnia: The Lion, the Witch ...","Siblings Lucy, Edmund, Susan and Peter step th..."
4,436270,Black Adam,"Nearly 5,000 years after he was bestowed with ..."


In [24]:
df2['tags'][0]

'Set more than a decade after the events of the first film, learn the story of the Sully family (Jake, Neytiri, and their kids), the trouble that follows them, the lengths they go to keep each other safe, the battles they fight to stay alive, and the tragedies they endure. ScienceFiction Adventure Action lossoflovedone dyinganddeath alienlife-form resurrection sequel dysfunctionalfamily alienplanet distantfuture adoptedchild rebirth familydynamics adoptedson strongervillain SamWorthington ZoeSaldaña SigourneyWeaver BritainDalton StephenLang JamesCameron'

In [25]:
df2['tags'] = df2['tags'].apply(lambda x:x.lower())

In [26]:
df2[df2['title'] == 'Iron Man']['tags'].values

array(['after being held captive in an afghan cave, billionaire engineer tony stark creates a unique weaponized suit of armor to fight evil. action sciencefiction adventure middleeast armsdealer malibu superhero basedoncomic aftercreditsstinger marvelcinematicuniverse(mcu) counterterrorism robertdowneyjr. terrencehoward jeffbridges gwynethpaltrow lesliebibb jonfavreau'],
      dtype=object)

In [27]:
df2[df2['title'] == 'Iron Man 2']['tags'].values

array(["with the world now aware of his dual life as the armored superhero iron man, billionaire inventor tony stark faces pressure from the government, the press and the public to share his technology with the military. unwilling to let go of his invention, stark, with pepper potts and james 'rhodey' rhodes at his side, must forge new alliances – and confront powerful enemies. adventure action sciencefiction technology malibu superhero basedoncomic revenge aftercreditsstinger marvelcinematicuniverse(mcu) breakout malibu,california,usa marvelcinematic superfiresuit prisoncellbreakout breakingout robertdowneyjr. gwynethpaltrow doncheadle scarlettjohansson samrockwell jonfavreau"],
      dtype=object)

## Feature Engineering

In [28]:
ps = PorterStemmer()

In [29]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [30]:
df2['tags'] = df2['tags'].apply(stem)

In [31]:
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
vectors = tfidf.fit_transform(df2['tags']).toarray()

In [32]:
tfidf.get_feature_names_out()

array(['000', '10', '100', '11', '12', '13', '14', '14thcenturi', '15',
       '15thcenturi', '16', '16thcenturi', '17', '17th', '17thcenturi',
       '18', '18th', '18thcenturi', '19', '1910', '1920', '1930', '1940',
       '1950', '1950s', '1960', '1960s', '1970', '1970s', '1980', '1987',
       '1990', '1999', '19th', '19thcenturi', '1stcenturi', '20', '2000',
       '2010', '2020', '2030', '2040', '20th', '21st', '24', '25', '30',
       '300', '35', '3d', '40', '50', '500', 'aaron', 'aaroneckhart',
       'abandon', 'abbiecornish', 'abduct', 'abigailbreslin', 'abil',
       'abl', 'aboard', 'abov', 'abroad', 'abus', 'academi', 'academy',
       'accept', 'access', 'accid', 'accident', 'accompani', 'accomplish',
       'account', 'accus', 'ace', 'achiev', 'acquaint', 'act', 'action',
       'actionhero', 'activ', 'activist', 'actor', 'actress', 'actual',
       'ad', 'adam', 'adamdriv', 'adamsandl', 'adapt', 'add', 'addict',
       'addit', 'adjust', 'admir', 'adolesc', 'adopt', 'a

In [33]:
similarity = cosine_similarity(vectors)

In [34]:
similarity[0]

array([1.        , 0.01629643, 0.01162083, 0.03627964, 0.04187116,
       0.01292546, 0.27422018, 0.00864755, 0.00617711, 0.00447048,
       0.00896355, 0.00734838, 0.0402158 , 0.01413741, 0.00822776,
       0.03670007, 0.03538521, 0.08232675, 0.        , 0.01353195,
       0.01879513, 0.        , 0.00754488, 0.24501331, 0.        ,
       0.09583774, 0.00489368, 0.02798949, 0.05736389, 0.01786376,
       0.        , 0.01379601, 0.12190763, 0.0039677 , 0.02356743,
       0.        , 0.00536289, 0.02533627, 0.        , 0.00636409,
       0.02429743, 0.        , 0.13966808, 0.01921508, 0.01945867,
       0.01141247, 0.02194398, 0.02215274, 0.01574684, 0.03400703,
       0.06634095, 0.05179739, 0.02504135, 0.0094581 , 0.0228113 ,
       0.01759281, 0.        , 0.01888723, 0.        , 0.0065954 ,
       0.0302446 , 0.00795879, 0.05195107, 0.04616638, 0.00704175,
       0.02081457, 0.04287296, 0.        , 0.        , 0.00401884,
       0.00492671, 0.0938099 , 0.03781929, 0.00543474, 0.00959

In [35]:
df2 = df2.reset_index(drop=True)

def recommend(movie):
    index = df2[df2['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:11]:
        print(df2.iloc[i[0]].title)

In [36]:
recommend("John Wick")

John Wick: Chapter 2
Xtreme
Sexy Beast
John Wick: Chapter 3 - Parabellum
John Wick: Chapter 4
How I Became a Gangster
Contract Killer
The Assignment
John Wick: Chapter 5
Fortress


In [37]:
import pickle

In [38]:
pickle.dump(df2.to_dict(), open('movies_dict.pkl', 'wb'))

In [39]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))